In [7]:
import os, sys
MYPROJECT = '/Users/ryuki/Projects/graduation_research'
sys.path.insert(0, MYPROJECT)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "settings.py")

import django
django.setup()

In [63]:
import pandas as pd

In [18]:
import MeCab
# import mojimoji
mecab = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

text = '解析したいテキスト'
mecab.parse('')#文字列がGCされるのを防ぐ

'EOS\n'

参考 https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja


In [29]:
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize c
hoonpus
    s = re.sub('[~∼∾〜〰～]', '', s)  # remove tildes
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

In [20]:
from review_based_recommender.models import Spot, Review, City
import django_pandas

In [21]:
from django_pandas.io import read_frame
reviews = Review.objects.all()
review_df = read_frame(reviews, fieldnames=['id', 'username', 'title', 'content', 'spot_id'], verbose=False)

In [22]:
review_df

,id,username,title,content,spot_id
0,1,niramechan,ボードウォーク閉鎖中,観音崎京急ホテルやその施設を利用するなら京急の馬堀海岸から送迎バスが出ています。ボードウォー...,23
1,2,yukof186,ハイキングでした…,美術館から足をのばして行きましたが、思ったより急な登りが沢山。軽い散歩のつもりでいったら、わ...,23
2,3,隆幸 末,楽しかった！,車は観音崎第一駐車場に止めました。海のすぐそばです。平日は無料のようです。駐車場から遊歩道を...,23
3,4,さすらいのとんしゅけ,海好きにはたまりません,ここのオススメは浦賀水道を通るお船を眺める事。一日にものすごい数の船が通るのです。タンカーや...,23
4,5,ya1201,海岸散歩,京急ホテル宿泊の時はいつも朝海岸散歩します。\n東京湾入口行き交う船を間近に見ながらの散歩は...,23
5,6,bjnoche,爽やかな潮風、自然での戯れ、モダンな施設、おいしい食事、、、癒しの場,観音崎は都心から最も近い自然浜のあるビーチリゾートとして人気があり、また地元の人にも海水浴や...,23
6,7,とよとよ25,なかなか楽しめます,横須賀の南にある岬で、中に入れる灯台や戦跡が残されており、観光地としてお勧めです。このあたり...,23
7,8,yoitoko137,東京湾に突き出した岬,三浦半島にある東京湾に突き出した岬で、東京湾を行き来する船舶がよく見える場所です。歴史のある...,23
8,9,Adam756,夏場はBBQ客がいっぱい,横須賀中央駅から京急バスで30分ぐらいで着きました。海水浴、BBQ、灯台を見に行ったり自然遊...,23
9,10,ずんだれ,横須賀を代表する景勝地,横須賀市歌は「白波は白波は岬にくだけ、光る風光る風大地に遊ぶ」という歌詞で始まりますが、まさ...,23


In [60]:
def get_reviews_list(df):
    res = df.loc[:]['content'].tolist()
    return res
df = review_df.groupby('spot_id').apply(get_reviews_list).to_frame('reviews').reset_index()

In [61]:
df

,spot_id,reviews
0,1,[最近、このあたりで仕事やプライベートの用事が多く、よく行くエリアです。ビルだらけではありま...
1,2,[ホタル観賞ができると聞き、行ってまいりました。30分ほど並びましたが、比較的スイスイ進み、...
2,3,[「桟橋」というと何か無機質な空間を思い浮かべますが、『横浜港大さん橋』ここは違います。\n...
3,4,[鎌倉に来たならやはり大仏は見ておくべきでしょう。周囲が緑に囲まれていて、まるで山奥にでも来...
4,5,[創建は住仁天皇（三世紀）と伝わる古社、相模国式内社です。\nケーブルカーの駅から数分で下社...
5,6,[エレベーターに乗り展望台へ。展望台を一回りし、屋外のデッキに出ました。屋外デッキの方が、風...
6,7,[航空路との関係で300mをわずかに切らねばならなかったのだそうだ。高さは296mとあべのハ...
7,8,[他のイベントに来て、空き時間にちょっと見るつもりでしたが、意外と広いし、中が迷路みたいにな...
8,9,[ここならゴーという硫黄泉と独特の匂い、７年寿命が伸びるという黒玉子、そして絶景の富士山が観...
9,10,[帆船日本丸とみなと博物館のあるエリアがメモリアルパークのようになっています。日本丸とみなと...


In [30]:
def reviews2tokens(reviews, stopwords):
    tokens = []
    for review in reviews:
        node = mecab.parseToNode(normalize_neologd(review))
        while node:
            word = node.surface
            if word in stopwords:
                continue
            pos = node.feature.split(",")[1]
            node = node.next
            tokens.append(word)
    return tokens

In [73]:
def reviews2tokensbysimpleparse(reviews, stopwords):
    tokens = []
    for review in reviews:
        for chunk in mecab.parse(review).splitlines()[:-1]:
            (surface, feature) = chunk.split('\t')
            if feature.startswith('名詞'):
                tokens.append(surface)
    return tokens

get stop word

In [31]:
import urllib.request
slothlib_path = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
slothlib_file  = urllib.request.urlopen(slothlib_path)
slothlib_stopwords = [line.decode("utf-8").strip() for line in slothlib_file if line.decode("utf-8").strip() != '']
slothlib_stopwords

↓時間かかるよ
参考： https://qiita.com/yukinoi/items/81a707c1317c97f5fdf9

In [74]:
# stopwords = ['\n']
# stopwords += slothlib_stopwords
# df['review_tokens'] = df['reviews'].apply(lambda doc: reviews2tokens(doc, stopwords))

**parse** faster than **parsetonode**

In [75]:
%%time
stopwords = ['\n','、','「','」']
stopwords += slothlib_stopwords
df['review_tokens'] = df['reviews'].apply(lambda doc: reviews2tokensbysimpleparse(doc, stopwords))

CPU times: user 54.6 s, sys: 1.63 s, total: 56.3 s
Wall time: 56.4 s


In [76]:
df

,spot_id,reviews,review_tokens
0,1,[最近、このあたりで仕事やプライベートの用事が多く、よく行くエリアです。ビルだらけではありま...,"[最近, あたり, 仕事, プライベート, 用事, エリア, ビル, だらけ, 駅, 間, ..."
1,2,[ホタル観賞ができると聞き、行ってまいりました。30分ほど並びましたが、比較的スイスイ進み、...,"[ホタル, 観賞, 30分, 待ち, ストレス, ホタル, 観賞, 区域, カメラ, 発光体..."
2,3,[「桟橋」というと何か無機質な空間を思い浮かべますが、『横浜港大さん橋』ここは違います。\n...,"[桟橋, 何, 無機質, 空間, 横浜港, 大さん橋, ここ, 国際, 客船, ターミナル,..."
3,4,[鎌倉に来たならやはり大仏は見ておくべきでしょう。周囲が緑に囲まれていて、まるで山奥にでも来...,"[鎌倉, 大仏, 周囲, 緑, 山奥, よう, 雰囲気, 過去, 何度, 鎌倉大仏, 久しぶ..."
4,5,[創建は住仁天皇（三世紀）と伝わる古社、相模国式内社です。\nケーブルカーの駅から数分で下社...,"[創建, 住仁, 天皇, 三, 世紀, 古社, 相模国, 式内社, ケーブルカー, 駅, 数..."
5,6,[エレベーターに乗り展望台へ。展望台を一回りし、屋外のデッキに出ました。屋外デッキの方が、風...,"[エレベーター, 展望台, 展望台, 一, 回, りし, 屋外, デッキ, 屋外, デッキ,..."
6,7,[航空路との関係で300mをわずかに切らねばならなかったのだそうだ。高さは296mとあべのハ...,"[航空路, 関係, 300m, の, そうだ, 高さ, 296m, あべのハルカス, 安定感..."
7,8,[他のイベントに来て、空き時間にちょっと見るつもりでしたが、意外と広いし、中が迷路みたいにな...,"[他, イベント, 空き, 時間, つもり, 中, 迷路, みたい, マリンタワー, 氷川丸..."
8,9,[ここならゴーという硫黄泉と独特の匂い、７年寿命が伸びるという黒玉子、そして絶景の富士山が観...,"[ここ, ゴー, 硫黄泉, 独特, ７, 年, 寿命, 黒玉子, 絶景, 富士山, お客様,..."
9,10,[帆船日本丸とみなと博物館のあるエリアがメモリアルパークのようになっています。日本丸とみなと...,"[帆船, 日, 本丸, みなと, 博物館, エリア, メモリアルパーク, よう, 日本丸, ..."


parallelize 参考 http://blog.adeel.io/2016/11/06/parallelize-pandas-map-or-apply/

In [77]:
# import numpy as np
# from multiprocessing import cpu_count, Pool
 
# cores = cpu_count() #Number of CPU cores on your system
# partitions = cores #Define as many partitions as you want
 
# def parallelize(data, func):
#     data_split = np.array_split(data, partitions)
#     pool = Pool(cores)
#     data = pd.concat(pool.map(func, data_split))
#     pool.close()
#     pool.join()
#     return data

In [78]:
# %time
# df = parallelize(df, lambda doc: reviews2tokensbysimpleparse(doc, stopwords))

In [79]:
# from dask import dataframe as dd
# from dask.multiprocessing import get
# from multiprocessing import cpu_count
# nCores = cpu_count()
# dd.from_pandas(df,npartitions=nCores).\
#    map_partitions(
#       lambda df : df.apply(
#          lambda doc: reviews2tokensbysimpleparse(doc, stopwords)), meta={'reviews': 'list'}).\
#    compute(get=get)

In [80]:
df

,spot_id,reviews,review_tokens
0,1,[最近、このあたりで仕事やプライベートの用事が多く、よく行くエリアです。ビルだらけではありま...,"[最近, あたり, 仕事, プライベート, 用事, エリア, ビル, だらけ, 駅, 間, ..."
1,2,[ホタル観賞ができると聞き、行ってまいりました。30分ほど並びましたが、比較的スイスイ進み、...,"[ホタル, 観賞, 30分, 待ち, ストレス, ホタル, 観賞, 区域, カメラ, 発光体..."
2,3,[「桟橋」というと何か無機質な空間を思い浮かべますが、『横浜港大さん橋』ここは違います。\n...,"[桟橋, 何, 無機質, 空間, 横浜港, 大さん橋, ここ, 国際, 客船, ターミナル,..."
3,4,[鎌倉に来たならやはり大仏は見ておくべきでしょう。周囲が緑に囲まれていて、まるで山奥にでも来...,"[鎌倉, 大仏, 周囲, 緑, 山奥, よう, 雰囲気, 過去, 何度, 鎌倉大仏, 久しぶ..."
4,5,[創建は住仁天皇（三世紀）と伝わる古社、相模国式内社です。\nケーブルカーの駅から数分で下社...,"[創建, 住仁, 天皇, 三, 世紀, 古社, 相模国, 式内社, ケーブルカー, 駅, 数..."
5,6,[エレベーターに乗り展望台へ。展望台を一回りし、屋外のデッキに出ました。屋外デッキの方が、風...,"[エレベーター, 展望台, 展望台, 一, 回, りし, 屋外, デッキ, 屋外, デッキ,..."
6,7,[航空路との関係で300mをわずかに切らねばならなかったのだそうだ。高さは296mとあべのハ...,"[航空路, 関係, 300m, の, そうだ, 高さ, 296m, あべのハルカス, 安定感..."
7,8,[他のイベントに来て、空き時間にちょっと見るつもりでしたが、意外と広いし、中が迷路みたいにな...,"[他, イベント, 空き, 時間, つもり, 中, 迷路, みたい, マリンタワー, 氷川丸..."
8,9,[ここならゴーという硫黄泉と独特の匂い、７年寿命が伸びるという黒玉子、そして絶景の富士山が観...,"[ここ, ゴー, 硫黄泉, 独特, ７, 年, 寿命, 黒玉子, 絶景, 富士山, お客様,..."
9,10,[帆船日本丸とみなと博物館のあるエリアがメモリアルパークのようになっています。日本丸とみなと...,"[帆船, 日, 本丸, みなと, 博物館, エリア, メモリアルパーク, よう, 日本丸, ..."


↓の参考：http://tdual.hatenablog.com/entry/2018/04/09/133000#gensim


In [83]:
docs = []
for index, row in df.iterrows():
    docs.append(row.review_tokens)

In [84]:
from gensim import corpora, models
d = corpora.Dictionary(docs)

In [90]:
# 使われている文書の数がno_belowより少ない単語を無視し、no_aboveの割合以上の文書に出てくる単語を無視している
d.filter_extremes(no_below=5, no_above=0.2)
d.compactify()

In [94]:
corpus = [d.doc2bow(w) for w in docs]
test_size = int(len(corpus) * 0.1)
test_corpus

NameError: name 'test_corpus' is not defined